In [ ]:
#@title Setup (Do NOT modify)
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/final
!pip install openai-clip

In [ ]:
#@title Data preprocessing (Train)
!python preproc_datasets_celeba_zip_train.py --source=./multimodal_celeba_hq.zip --dest train_data_6cap.zip --emb_dim 512 --transform=center-crop --width=256 --height=256

In [ ]:
#@title Data preprocessing (Test)
!python preproc_datasets_celeba_zip_test.py --source=./multimodal_celeba_hq.zip --dest test_data_6cap.zip --emb_dim 512 --transform=center-crop --width=256 --height=256

In [ ]:
#@title Train

train_data = "sample_train.zip"#@param {"type": "string"}
batch_size = 24 #@param {"type": "integer"}
num_epochs = 10 #@param {"type": "integer"}
learning_rate = 0.0002 #@param {"type": "number"}
report_interval = 50 #@param {"type": "integer"}
noise_dim = 100 #@param {"type": "integer"}
projection_dim = 128 #@param {"type": "integer"}
clip_embedding_dim = 512 #@param {"type": "integer"}
checkpoint_path = "model_exp1" #@param {"type": "string"}
result_path = "images_exp1" #@param {"type": "string"}
use_uncond_loss = True #@param {"type": "boolean"}
use_contrastive_loss = True #@param {"type": "boolean"}
num_stage = 3 #@param {"type": "integer"}
resume_checkpoint_path = "None" #@param {"type": "string"}
resume_epoch = -1 #@param {"type": "integer"}

test_cmd = f'''python main.py \
    --train_data "{train_data}" \
    --batch_size {batch_size} \
    --num_epochs {num_epochs} \
    --learning_rate {learning_rate} \
    --report_interval {report_interval} \
    --noise_dim {noise_dim} \
    --projection_dim {projection_dim} \
    --clip_embedding_dim {clip_embedding_dim} \
    --checkpoint_path "{checkpoint_path}" \
    --result_path "{result_path}" \
    --num_stage {num_stage} \
    --resume_epoch {resume_epoch} \
    '''
if use_uncond_loss:
    test_cmd += "--use_uncond_loss "
if use_contrastive_loss:
    test_cmd += "--use_contrastive_loss "
if resume_checkpoint_path != "None":
    test_cmd += f'''--resume_checkpoint_path "{resume_checkpoint_path}"'''

with open('./train_script.sh', 'w') as file:
    file.write(test_cmd)

!bash train_script.sh

In [ ]:
#@title Test (Generate image)

prompt = "The woman is young and has blond hair, and arched eyebrows."#@param {"type": "string"}
load_epoch = 10 #@param {"type": "integer"}
checkpoint_path = "model_exp1" #@param {"type": "string"}

test_cmd = f'''python generate_image.py \
    --prompt "{prompt}" \
    --load_epoch {load_epoch} \
    --checkpoint_path "{checkpoint_path}"
    '''

with open('./test_script.sh', 'w') as file:
    file.write(test_cmd)

!bash test_script.sh


from IPython.display import Image
import os
img_64 = Image(os.path.join("result_64.png"))
display(img_64)
img_128 = Image(os.path.join("result_128.png"))
display(img_128)
img_256 = Image(os.path.join("result_256.png"))
display(img_256)